In [1]:
import torch

In [2]:
from torch.utils.data import Dataset

In [3]:
import json
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import missingno as msno

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
from keras.preprocessing import text
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer

In [7]:

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer


c:\Users\Dell\anaconda3\envs\BERT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename = 'intents.json'

intents = load_json_file(filename)

In [9]:
def create_df():
    df = pd.DataFrame({
        'Pattern' : [],
        'Tag' : []
    })
    
    return df

df = create_df()
df

,Pattern,Tag


In [10]:
def extract_json_info(json_file, df):
    
    for intent in json_file['intents']:
        
        for pattern in intent['patterns']:
            
            sentence_tag = [pattern, intent['tag']]
            df.loc[len(df.index)] = sentence_tag
                
    return df

df = extract_json_info(intents, df)
df.head()

,Pattern,Tag
0,Hello,greeting
1,Hi,greeting
2,Hey,greeting
3,Good morning,greeting
4,Good evening,greeting


In [11]:
df2 = df.copy()
df2.head()

,Pattern,Tag
0,Hello,greeting
1,Hi,greeting
2,Hey,greeting
3,Good morning,greeting
4,Good evening,greeting


In [12]:
def print_shape_df(df, ds_name="df"):
    print(f"{ds_name} dataset has {df.shape[0]} rows and {df.shape[1]} columns")
    
print_shape_df(df, "Chatbot")

Chatbot dataset has 81 rows and 2 columns


In [13]:
def print_dfInfo(df, ds_name="df"):
    print(f"The info of {ds_name} dataset\n")
    print(df.info())
    
print_dfInfo(df, "Chatbot")

The info of Chatbot dataset

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 80
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pattern  81 non-null     object
 1   Tag      81 non-null     object
dtypes: object(2)
memory usage: 1.9+ KB
None


In [14]:
def num_classes(df, target_col, ds_name="df"):
    print(f"The {ds_name} dataset has {len(df[target_col].unique())} classes")
    
num_classes(df, 'Tag', "Chatbot")

The Chatbot dataset has 21 classes


In [15]:
def check_null(df, ds_name='df'):
    print(f"Null Values in each col in the {ds_name} dataset:\n")
    print(df.isnull().sum())
    
check_null(df, "Chatbot")

Null Values in each col in the Chatbot dataset:

Pattern    0
Tag        0
dtype: int64


In [16]:
df2.head()

,Pattern,Tag
0,Hello,greeting
1,Hi,greeting
2,Hey,greeting
3,Good morning,greeting
4,Good evening,greeting


In [17]:
labels = df2['Tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels

['greeting',
 'goodbye',
 'thanks',
 'help',
 'admission_info',
 'academic_calendar',
 'contact_info',
 'important_dates',
 'facility_info',
 'latest_news',
 'locate_results',
 'locate_admissions',
 'locate_academic_calendar',
 'locate_notices',
 'locate_contact_info',
 'notice_board',
 'job_openings',
 'tender_notices',
 'events',
 'student_welfare',
 'vision_mission']

In [18]:
num_labels = len(labels)
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}

In [19]:
id2label

{0: 'greeting',
 1: 'goodbye',
 2: 'thanks',
 3: 'help',
 4: 'admission_info',
 5: 'academic_calendar',
 6: 'contact_info',
 7: 'important_dates',
 8: 'facility_info',
 9: 'latest_news',
 10: 'locate_results',
 11: 'locate_admissions',
 12: 'locate_academic_calendar',
 13: 'locate_notices',
 14: 'locate_contact_info',
 15: 'notice_board',
 16: 'job_openings',
 17: 'tender_notices',
 18: 'events',
 19: 'student_welfare',
 20: 'vision_mission'}

In [20]:
label2id

{'greeting': 0,
 'goodbye': 1,
 'thanks': 2,
 'help': 3,
 'admission_info': 4,
 'academic_calendar': 5,
 'contact_info': 6,
 'important_dates': 7,
 'facility_info': 8,
 'latest_news': 9,
 'locate_results': 10,
 'locate_admissions': 11,
 'locate_academic_calendar': 12,
 'locate_notices': 13,
 'locate_contact_info': 14,
 'notice_board': 15,
 'job_openings': 16,
 'tender_notices': 17,
 'events': 18,
 'student_welfare': 19,
 'vision_mission': 20}

In [21]:
df2['labels'] = df2['Tag'].map(lambda x: label2id[x.strip()])
df2.head()

,Pattern,Tag,labels
0,Hello,greeting,0
1,Hi,greeting,0
2,Hey,greeting,0
3,Good morning,greeting,0
4,Good evening,greeting,0


In [22]:
X = list(df2['Pattern'])
X[:5]

['Hello', 'Hi', 'Hey', 'Good morning', 'Good evening']

In [23]:
y = list(df2['labels'])
y[:5]

[0, 0, 0, 0, 0]

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 123)

In [25]:
model_name = "bert-base-uncased"
max_len = 256

tokenizer = BertTokenizer.from_pretrained(model_name, 
                                          max_length=max_len)

model = BertForSequenceClassification.from_pretrained(model_name, 
                                                      num_labels=num_labels, 
                                                      id2label=id2label, 
                                                      label2id = label2id)

c:\Users\Dell\anaconda3\envs\BERT\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification f

In [26]:
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

In [27]:
full_data = tokenizer(X, truncation=True, padding=True)

In [28]:
class DataLoader(Dataset):
    
    def __init__(self, encodings, labels):
        
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
               
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [29]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)

In [30]:
fullDataLoader = DataLoader(full_data, y_test)

In [31]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [32]:
training_args = TrainingArguments(
    output_dir='./output', 
    do_train=True,
    do_eval=True,
    num_train_epochs=100,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=16,
    warmup_steps=100,                
    weight_decay=0.05,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    load_best_model_at_end=True
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=test_dataloader,            
    compute_metrics= compute_metrics
)

NameError: name 'training_args' is not defined

In [33]:
trainer.train()

NameError: name 'trainer' is not defined

In [34]:
q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]

pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

NameError: name 'trainer' is not defined

In [35]:
def predict(text):
    
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [36]:
text = "Hello"
predict(text)

AssertionError: Torch not compiled with CUDA enabled

In [37]:
model_path = "chatbot"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

NameError: name 'trainer' is not defined

In [38]:
pip install xformers

In [ ]:
model_path = "chatbot"


model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
chatbot= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
chatbot("Hello")

In [ ]:
def chat(chatbot):
    
    print("Chatbot: Hi! I am your virtual assistance,Feel free to ask, and I'll do my best to provide you with answers and assistance..")
    print("Type 'quit' to exit the chat\n\n")
    
    text = input("User: ").strip().lower()
    
    while(text != 'quit'):

        score = chatbot(text)[0]['score']
        
        if score < 0.8:
            print("Chatbot: Sorry I can't answer that\n\n")
            text = input("User: ").strip().lower()
            continue
        
        label = label2id[chatbot(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])
        
        print(f"Chatbot: {response}\n\n")
            
        text = input("User: ").strip().lower()

In [ ]:
chat(chatbot)